# Jupyter Notebook to play around with MOKU:GO

## nakago2 : 129.206.180.142

In [1]:
from moku.instruments import PIDController
from moku.instruments import Oscilloscope

import matplotlib.pyplot as plt
import numpy as np

import datetime

%matplotlib

Matplotlib is building the font cache; this may take a moment.


Using matplotlib backend: GTK3Agg


/home/imdippel/.local/lib/python3.9/site-packages/ipykernel/gui/gtk3embed.py:59: RuntimeWarning: coroutine 'Kernel.do_one_iteration' was never awaited
  self.kernel.do_one_iteration()


## Moku:Go (nakago2)

In [ ]:
# This example demonstrates how you can configure the Oscilloscope instrument,
# and view triggered time-voltage data frames in real-time.
#
# (c) 2021 Liquid Instruments Pty. Ltd.

print('START : ' + datetime.datetime.now().strftime("%d/%m/%Y, %H:%M:%S"))
def on_close(event):
    print('Closed Figure!')
    i.relinquish_ownership()
    
# Connect to your Moku by its ip address using Oscilloscope('192.168.###.###')
# or by its serial number using Oscilloscope(serial=123)
i = Oscilloscope('129.206.180.142', force_connect=True)

try:
    # Trigger on input Channel 1, rising edge, 0V 
    i.set_trigger(type='Edge', source='Input1', level=0)

    # View +-5usec, i.e. trigger in the centre
    i.set_timebase(-5e-6, 5e-6)

    # Generate an output sine wave on Channel 1, 1Vpp, 1MHz, 0V offset
    i.generate_waveform(1, 'DC', amplitude=1)
    i.generate_waveform(2, 'Sine', amplitude=1, frequency=1e6)

    # Set the data source of Channel 1 to be Input 1
    i.set_source(1, 'Input1')

    # Set the data source of Channel 2 to the generated output sinewave
    i.set_source(2, 'Input2')


    # Get initial data frame to set up plotting parameters. This can be done
    # once if we know that the axes aren't going to change (otherwise we'd do
    # this in the loop)
    data = i.get_data()
    print(data.keys())

    # Set up the plotting parameters
    plt.ion()
    plt.show()
    plt.grid(b=True)
    plt.ylim([-1, 1])
    plt.xlim([data['time'][0], data['time'][-1]])

    line1, = plt.plot([], label='channel 1')
    line2, = plt.plot([], label='channel 2')

    # Configure labels for axes
    ax = plt.gca()
    
    # This loops continuously updates the plot with new data
    while True:
        # Get new data
        data = i.get_data()

        # Update the plot
        line1.set_ydata(data['ch1'])
        line2.set_ydata(data['ch2'])
        line1.set_xdata(data['time'])
        line2.set_xdata(data['time'])
        
        ax.set_title(datetime.datetime.now().strftime("%d/%m/%Y, %H:%M:%S"))
        ax.legend()

        plt.pause(0.001)
        
        fig = plt.gcf()  # get current figure
        fig.canvas.mpl_connect('close_event', on_close)
        
except Exception as e:
    print(f'Exception occurred: {e}')
except KeyboardInterrupt:
    print('Interrupt!')
finally:
    # Close the connection to the Moku device
    # This ensures network resources and released correctly
    i.relinquish_ownership()
    print('Ownership relinquished!')

In [ ]:
# Connect to your Moku by its ip address using Oscilloscope('192.168.###.###')
# or by its serial number using Oscilloscope(serial=123)
i = Oscilloscope('129.206.180.142', force_connect=True)

# Generate an output sine wave on Channel 1, 1Vpp, 1MHz, 0V offset
i.generate_waveform(1, 'Ramp', amplitude=2)
# i.generate_waveform(2, 'Sine', amplitude=1, frequency=1e6)

In [ ]:
i.relinquish_ownership()